In [ ]:
# !pip install transformers
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
# !pip install sentence-transformers
# !pip install datasets

In [1]:
import numpy as np
import pandas as pd
import ast
import warnings
import scipy
import matplotlib.pyplot as plt
import spacy
import difflib
import tqdm
import json
import pickle
import logging
import itertools
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from datasets import Dataset
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

/Users/gke/py312env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-25 22:56:36,853 : INFO : PyTorch version 2.3.1 available.


In [12]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
tokenizer = model.tokenizer

dfn = pd.read_csv('lmsys-chatbot-arena/aux_files/dfn.csv')

dfn.head(3)

2024-06-26 15:40:13,459 : INFO : Use pytorch device_name: mps
2024-06-26 15:40:13,460 : INFO : Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
/Users/gke/py312env/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


,id,prompt,model_a_answer,model_b_answer,winner
0,30192,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",A
1,53567,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,B
2,53567,How can I get both of them as quick as possibl...,If you want to get both a marriage license and...,"In California, here are the general steps to o...",B


In [35]:
len(tokenizer(dfn.iloc[5]['model_a_answer']).encodings[0].tokens)

659

In [19]:
unique_ids = dfn.id.unique()
tagger = spacy.load("en_core_web_lg")

In [56]:
doc = tagger("I am testing. Just to see NYC")
for token in doc:
    print(token.pos_ , token.dep_ , token.text, token.is_stop)

PRON nsubj I True
AUX aux am True
VERB ROOT testing False
PUNCT punct . False
ADV advmod Just True
PART aux to True
VERB ROOT see True
PROPN dobj NYC False


In [50]:
def reduce_text(txt, tagger , limit , target_tags = ["PROPN" , "NOUN" , "VERB" , "ADP" , "PRON" , "AUX" , "ADJ"]):
    reconstructed_str = ""
    doc = tagger(txt)
    for token in txt:
        if token.pos_ == "PROPN" or token.pos_:
            if len(reconstructed_str):
                reconstructed_str = token.text
            else:
                reconstructed_str += " " + token.text
        if token.

In [37]:
model.max_seq_length

256

In [41]:
# filter out by tag, check first if string can fit in tokenizer
seq_overhead = 3
max_seq = model.max_seq_length - seq_overhead
id_list = []
anchors = []
positives = []
for _id in tqdm.tqdm(unique_ids):
    id_df = dfn[dfn.id == _id]
    for row in id_df.iterrows():
        id_list.append(_id)
        
        prompt = row[1]['prompt']
        model_a_answer = row[1]['model_a_answer']
        model_b_answer = row[1]['model_b_answer']
        winner = row[1]['winner']

        prompt_len = len(tokenizer(prompt).encodings[0].tokens)
        model_a_answer_len = len(tokenizer(model_a_answer).encodings[0].tokens)
        model_b_answer_len = len(tokenizer(model_b_answer).encodings[0].tokens)

        if winner == "A":
            if model_a_answer_len > max_seq:
                
                for token in doc:
            pass
        if winner == "B":
            pass
        if winner == "N":
            pass
    
    break

  0%|                                                 | 0/56759 [00:00<?, ?it/s]
